<h1> Evaluating different chunking strategies, chunk sizing and overlap using RAGA framework: Claude 3 Sonnet on Bedrock for LLM, FAISS for Vector Store and RAGA for Evaluation   <h1>

#### Identifying  documents best chunking Strategy is key to increase the accuracy of retrieved responses with solutions involving RAG and LLM models. The following notebooks implements a framework to evaluate different chuncking strategies. The goal is not to identify the best chunking strategy in the context of the example from the notebook rather to provide a tool to implement this evaluation for real case example 
The notebook follow the following steps
1. Install required libraries
2. Create a bedrock Client
3. Load documents example in a local folder
4. Use a custom function "textsplitterStrategy" with 3 params : Strategy , chunk size and overlap
5. run the function for 5 chunking strategies and load the documents in FAISS (5 stores)
6. Use Claude3 Sonnet and RAGA to evaluate  context_recall,context_precision,  answer_relevancy
7. plot the evaluation results

    

<h4>Install required libraries<h4>

In [ ]:
!pip3  install  -r requirements.txt 

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


<h4>Create a bedrock client using boto3<h4>

In [ ]:
# 2.Create a bedrock Client
import json
import os
import boto3
import pprint
import random
from retrying import retry
import nltk
nltk.download('punkt')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region_name,
)




In [ ]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

<h3> Use a custom function "textsplitterStrategy" with 3 params : Strategy , chunk size and overlap</h3>

In [ ]:
# 4. Use a custom function "textsplitterStrategy" with 3 params : Strategy , chunk size and overlap
#textsplitter function : 
# param 1 : strategy   (LatexTextSplitter== 2 ), (NLTKTextSplitter== 1) ,(MarkdownTextSplitter' == 3)  , (RecursiveCharacterTextSplitter == 4),  strategytext='SpacyTextSplitter'
#param 2: chunksize 
#param 3: overlapp

#reminder of each Strategy 

 # Markdown: Markdown is a lightweight markup language commonly used for formatting text. By recognizing the Markdown syntax (e.g., headings, lists, and code blocks), 
 # Recursive chunking divides the input text into smaller chunks in a hierarchical and iterative manner using a set of separators.
 #LaTex: LaTeX is a document preparation system and markup language often used for academic papers and technical documents. chunks respects the logical organization of the content (e.g., sections, subsections, and equations
 # NLTK is a free, open-source library for advanced Natural Language Processing (NLP) in Python. It can help simplify textual data and gain in-depth information from input messages.
# Spacy, another powerful NLP library, provides a sentence tokenization function that relies heavily on linguistic rules. It is a similar approach to NLTK.

def textsplitterStrategy(strategy, chunksize , overlap):
    from langchain.text_splitter import LatexTextSplitter
    from langchain.text_splitter import MarkdownTextSplitter
    from langchain.text_splitter import SpacyTextSplitter
    from langchain.text_splitter import NLTKTextSplitter
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    import nltk
    import numpy as np
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.document_loaders import PyPDFLoader
    documents = []
    docs =[]

    for idx, file in enumerate(filenames):
        loader = PyPDFLoader(data_root + file)
        document = loader.load()
        for document_fragment in document:
            document_fragment.metadata = metadata[idx]
        
        documents += document

    if (strategy == 2):
        text_splitter = LatexTextSplitter(
            chunk_size =  chunksize,
            chunk_overlap  = overlap,
        )
        docs = text_splitter.split_documents(documents)
  
    if (strategy == 1):
        text_splitter = NLTKTextSplitter(
            chunk_size =  chunksize,
            chunk_overlap  = overlap,
         
        )
        docs = text_splitter.split_documents(documents) 
    if (strategy == 3):
        text_splitter = MarkdownTextSplitter(
            chunk_size =  chunksize,
            chunk_overlap  = overlap,
        )
        docs = text_splitter.split_documents(documents)
    if (strategy == 4):
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size =  chunksize,
            chunk_overlap  = overlap,
           # is_separator_regex=False,
            separators=["\n\n", "\n", ' ', ''],
            length_function = len
            
        )
        docs = text_splitter.split_documents(documents)  
    if (strategy == 5):
        text_splitter = SpacyTextSplitter(
            chunk_size =  chunksize,
            chunk_overlap  = overlap
           #separator = ['\n', '=', ',', '&']
        )
        docs = text_splitter.split_documents(documents) 
    avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
    print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
    print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
    print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')         
    return docs 

In [ ]:
#get stragtey label to dispaly when plotting
def Strategy_label(strategy):
    strategytext=""
    if (strategy == 2):
        strategytext='LatexTextSplitter'
    if (strategy == 1):
       strategytext='NLTKTextSplitter'
    if (strategy == 3):
          strategytext='MarkdownTextSplitter'
    if (strategy == 4):
        strategytext='RecursiveCharacterTextSplitter'
    if (strategy == 5):
        strategytext='SpacyTextSplitter'
    return strategytext 

In [ ]:
!python3   -m spacy download en_core_web_sm --quiet

### Provide as parameter one of strategies below : and load chunk in a list of chunked documents
#### 1 : NLTKTextSplitter
#### 2 :LatexTextSplitter
#### 3:MarkdownTextSplitter
#### 4:RecursiveCharacterTextSplitter
#### 5:SpacyTextSplitter
#### Provide chunk size and overlap for docs with different strategies

In [ ]:
#  Provide parameter one of those strategies: and load chunk in a list fo chunked documents
#1: NLTKTextSplitter
#2: LatexTextSplitter
#3: MarkdownTextSplitter
#4: RecursiveCharacterTextSplitter
#5: SpacyTextSplitter
#vectors for docs with different strategies
#  chunk size as paramter : 1000 here for example purpose
#  pass overlap as paramter : 150 here for example purpose  
my_docs = []
my_strategies =[]
strategy=""
strategytext=""
for x in [1,2,3,4,5]:
    strategy=Strategy_label(x)  
    print (strategy)
    prefixed_name = "docs_" + str(x)
    prefixed_name =  textsplitterStrategy(x,1000,150)
    my_docs.append(prefixed_name)
    my_strategies.append(strategy)

### Bedrock embedding model : amazon.titan-embed-text-v1

In [ ]:
from langchain.embeddings import BedrockEmbeddings
bedrock_client = boto3.client(service_name='bedrock-runtime', 
                              region_name='us-east-1')
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1",
                                       client=bedrock_client)

#### Run the function for 5 chunking strategies and load the documents in FAISS (5 stores)
#### the execution of this loop  would requires 2-3 minutes to embed chunks and load them as vector in FAISS
#### 5 strategies , 5 chunked documents (same documents chunked using different strategies) and loaded to FAISS: One store per Strategy¶


In [ ]:
from langchain.vectorstores import FAISS 
db_vector =[]
for x in [1,2,3,4,5]:
    db  = FAISS.from_documents(
        my_docs[x-1],
        bedrock_embeddings,
    )
    db_vector.append(db)

#### Test the Vector Stores : use db_vector[x] , where x = 0,1,2,3,4
##### 1: NLTKTextSplitter
##### 2: LatexTextSplitter
##### 3: MarkdownTextSplitter
##### 4: RecursiveCharacterTextSplitter
##### 5: SpacyTextSplitter

In [ ]:
#test on the list of DBs
query = "What is Amazon doing in the field of Generative AI?"
retriever = db_vector[2].as_retriever()
retrievalResults  = retriever.invoke(query)
print(retrievalResults)

#### Define LLM prompt

In [ ]:
contexts = []
#LLM prompt
prompt = f"""
Human: You are an advisor AI system, and provides answers to questions. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific.

Assistant:"""

#### Define LLM paramters including max_tokens size, temperature and top-p

In [ ]:

# payload with model paramters
messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(contexts, query)}]}]
sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 2000,
    "messages": messages,
    "temperature": 0.5,
    "top_p": 1
        }  )


#### install boto3 required version , uncomment if errors only 

#### Define LLM used for test : on our case model used for test anthropic.claude-3-sonnet-20240229-v1:0 on Bedrock 

In [ ]:
from langchain_aws import ChatBedrock
from langchain.llms.bedrock import Bedrock
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=sonnet_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('content')[0]['text']
llm = ChatBedrock(model_id=modelId, 
                  client=bedrock_client)

#### Define RAG Chain evaluator 

In [ ]:
#define RAG Chain evaluator 

from langchain.prompts import ChatPromptTemplate ,PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

PROMPT_TEMPLATE = """
    Human: You are an advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {question}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt2 = PromptTemplate.from_template(PROMPT_TEMPLATE)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [ ]:
!pip3 install pydantic==1.10.17 --force-reinstall  

### Define RAGAS metrics LLM  : Context_recall, context_precision, answer_relevancy

##### Context_recall : Context recall measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.

##### Context Precision is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks.

##### Answer Correctness¶ : The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness.

###### https://docs.ragas.io/en/latest/concepts/metrics/index.html

In [ ]:
from ragas.metrics import (
    context_precision,
    faithfulness,
    context_recall,
    answer_relevancy

)

from ragas.metrics.critique import harmfulness
# list of metrics we're going to use
metrics = [
   # faithfulness,
    context_recall,
    context_precision,
  #  harmfulness,
    answer_relevancy
    
]

#### This function will be called by Store in order to evaluate RAGA metrics
#### This function ensure that you will use the right retriever to relevant to store i e strategy
#### It also sends traces to langefuse for tracability for latency calculation 
#### Questions and Ground_truths are used from other AWS github examples 

In [ ]:
def evaluatewithReriever (retriever_name,i):
    questions = ["How many days has Amazon asked employees to come to work in office?", 
                 "By what percentage did AWS revenue grow year-over-year in 2022?",
                 "Compared to Graviton2 processors, what performance improvement did Graviton3 chips deliver according to the passage?",
                 "Which was the first inference chip launched by AWS according to the passage?",
                 "According to the context, in what year did Amazon's annual revenue increase from $245B to $434B?",
                 "What did the president say about Intel's CEO?",
                ]
    ground_truths = [["Amazon has asked corporate employees to come back to office at least three days a week beginning May 2022."],
                    ["AWS had a 29% year-over-year ('YoY') revenue in 2022 on $62B revenue base."],
                    ["In 2022, AWS delivered their Graviton3 chips, providing 25% better performance than the Graviton2 processors."],
                    ["AWS launched their first inference chips (“Inferentia”) in 2019, and they have saved companies like Amazon over a hundred million dollars in capital expense."],
                    ["Amazon's annual revenue increased from $245B in 2019 to $434B in 2022."],
                    ["The president asked Congress to pass proven measures to reduce gun violence."]]
    
    answers = []
    contexts = []
    ragas_scores = []
    #print ("retriever_name" ,retriever_name)
    # Inference
    for query in questions:
      answers.append(rag_chain.invoke(query))
      contexts.append([docs.page_content for docs in retriever_name.get_relevant_documents(query)])
 
    answer = rag_chain.invoke(query)
     
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset

#### Use Claude3 sonnet,  RAGA to evaluate context_recall,context_precision, answer_relevancy, 
#### Load all results in dataframes in order to plot results 
#### Execution time will take between 5 - 10 minutes

In [ ]:
# RAGA tested on the 5 FAISS strors  
from datasets import Dataset
from ragas import evaluate
import nest_asyncio  # CHECK NOTES
df_vector= []
# NOTES: Only used when running on a jupyter notebook, otherwise comment or remove this function.
nest_asyncio.apply()

for i in [0,1,2,3,4]:
    retriever = db_vector[i].as_retriever()
    dataset = evaluatewithReriever(retriever,i)
    result = evaluate(
        dataset,
        metrics=metrics,
        llm=llm,
        embeddings=bedrock_embeddings,
    )
    df = result.to_pandas()
    df_vector.append(df)

#### Plot results per strategy

In [ ]:

for i in [0,1,2,3,4]:
    ax = df_vector[i].plot.bar(figsize=(10,3) , title=my_strategies[i])
    for container in ax.containers:
        ax.bar_label(container)



### Plot Results as aggregated Score in one Dashboard

In [ ]:
## create average DF Per strategy 
import pandas as pd
import matplotlib.pyplot as plt
dict = {'strategy' :[0], 'context_recall':[0],'context_precision':[0] , 'answer_relevancy':[0] }
averagedf=pd.DataFrame(dict)
#averagedf= pd.DataFrame(columns = ['Strategy', 'context_recall','context_precision' , 'answer_relevancy' ])
for i in [0,1,2,3,4]:
   df2 = {'strategy': my_strategies [i],'context_recall': df_vector[i]["context_recall"].mean() ,  'context_precision': df_vector[i]["context_precision"].mean()
          , 'answer_relevancy': df_vector[i]["answer_relevancy"].mean() }
   averagedf = averagedf._append(df2, ignore_index = True)
   
averagedf  = averagedf.drop(averagedf [(averagedf.strategy == '0')].index)
averagedf = averagedf.drop([0])
print (averagedf)

ax = averagedf.plot.bar(x='strategy' ,figsize=(20,10) , title= "Chunking Strategy Evaluation") 
for container in ax.containers:
        ax.bar_label(container)
plt.show()
ax2 = averagedf.plot('strategy' ,figsize=(20,10) , title= "Chunking Strategy Evaluation") 
for container in ax2.containers:
        ax2.bar_label(container)
plt.show()